In [56]:
import tensorflow as tf
from tensorflow import keras
import pickle
import pandas as pd
import numpy as np

In [57]:
## Load the trained model, scaler pickle , and onehot
model = keras.models.load_model("model.h5")

## Load the encoder and scaler
with open("label_encoder_gender.pkl", "rb") as f:
    label_encoder_gender = pickle.load(f)

with open("onehot_encoder_geo.pkl", "rb") as f:
    onehot_encoder_geo = pickle.load(f)

with open("scaler.pkl", "rb") as f:
    scaler = pickle.load(f)

In [58]:
## Example input data
input_data = {
    "CreditScore": 600,
    "Geography": "France",
    "Gender": "Male",
    "Age": 40,
    "Tenure": 3,
    "Balance": 60000,
    "NumOfProducts": 2,
    "HasCrCard": 1,
    "IsActiveMember": 1,
    "EstimatedSalary": 50000,

}

In [61]:
## Transfer the input_data
### One-hot endcode 'Geography'
geography_encoded = onehot_encoder_geo.transform([[input_data["Geography"]]])
geography_encoded_df = pd.DataFrame(geography_encoded, columns=onehot_encoder_geo.get_feature_names_out(["Geography"]))
geography_encoded_df

/Users/hisl/miniconda3/envs/GenAICourse/lib/python3.12/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [64]:
## Combine on-hot coded columns in input data
input_df = pd.DataFrame([input_data])

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,France,Male,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [65]:
## Encode categorical variables
input_df["Gender"] = label_encoder_gender.transform([input_data["Gender"]])
input_df = pd.concat([input_df.drop("Geography", axis=1), geography_encoded_df], axis=1)
input_df

/Users/hisl/miniconda3/envs/GenAICourse/lib/python3.12/site-packages/sklearn/preprocessing/_label.py:129: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [66]:
## Scale the input data
input_df_scaled = scaler.transform(input_df)
input_df_scaled

array([[-0.53598516,  0.91324755,  0.10479359, -0.69539349, -0.25781119,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

In [67]:
## Predict churn
prediction = model.predict(input_df_scaled)
prediction

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step


array([[0.02841503]], dtype=float32)

In [68]:
if prediction > 0.5:
    print("The customer is likely to churn")
else:
    print("The customer is not likely to churn")

The customer is not likely to churn
